In [ ]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
from tqdm.notebook import tqdm

import joblib
import lightgbm as lgb

#### LGBM from https://www.kaggle.com/hengzheng/lightgbm-5folds-baseline
#### NN from https://www.kaggle.com/leolu1998/ubiquant-nn-baseline-train-infer

In [ ]:
lgb_model_0 = joblib.load('../input/ubiquant-lightgbm-5folds/lgb_0.pkl')
lgb_model_1 = joblib.load('../input/ubiquant-lightgbm-5folds/lgb_1.pkl')
lgb_model_2 = joblib.load('../input/ubiquant-lightgbm-5folds/lgb_2.pkl')
lgb_model_3 = joblib.load('../input/ubiquant-lightgbm-5folds/lgb_3.pkl')
lgb_model_4 = joblib.load('../input/ubiquant-lightgbm-5folds/lgb_4.pkl')

In [ ]:
lgb_0 = joblib.load('../input/lightgbm-5folds-baseline/lgb_0.pkl')
lgb_1 = joblib.load('../input/lightgbm-5folds-baseline/lgb_1.pkl')
lgb_2 = joblib.load('../input/lightgbm-5folds-baseline/lgb_2.pkl')
lgb_3 = joblib.load('../input/lightgbm-5folds-baseline/lgb_3.pkl')
lgb_4 = joblib.load('../input/lightgbm-5folds-baseline/lgb_4.pkl')

In [ ]:
from keras.models import load_model
nn0 = load_model('../input/ubiquant-nn-training/nn_0.h5')
nn1 = load_model('../input/ubiquant-nn-training/nn_1.h5')
nn2 = load_model('../input/ubiquant-nn-training/nn_2.h5')
nn3 = load_model('../input/ubiquant-nn-training/nn_3.h5')
nn4 = load_model('../input/ubiquant-nn-training/nn_4.h5')

In [ ]:
nn_0 = load_model('../input/ump-nn-model-zscore-logcosh/nn_0.h5')
nn_1 = load_model('../input/ump-nn-model-zscore-logcosh/nn_1.h5')
nn_2 = load_model('../input/ump-nn-model-zscore-logcosh/nn_2.h5')
nn_3 = load_model('../input/ump-nn-model-zscore-logcosh/nn_3.h5')
nn_4 = load_model('../input/ump-nn-model-zscore-logcosh/nn_4.h5')

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, scale
import pandas as pd
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
scaler = MinMaxScaler(feature_range=(-1, 1))  
 
features_to_consider = test.drop(['row_id','time_id','investment_id'],axis=1).columns

scaler.fit_transform(test[features_to_consider].values)   

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id'], axis=1, inplace=True)
    tt = scaler.transform(test_df[features_to_consider].values)
    ss = scale(test_df[features_to_consider].values)
    pred_nn0 = nn0.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn1 = nn1.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn2 = nn2.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn3 = nn3.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn4 = nn4.predict([test_df['investment_id'],tt]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    
    pred_nn_0 = nn_0.predict([test_df['investment_id'],ss]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn_1 = nn_1.predict([test_df['investment_id'],ss]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn_2 = nn_2.predict([test_df['investment_id'],ss]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn_3 = nn_3.predict([test_df['investment_id'],ss]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    pred_nn_4 = nn_4.predict([test_df['investment_id'],ss]).reshape(1,-1)[0].clip(-1.2e2,1.2e2)/100
    
    pred_0 = lgb_model_0.predict(test_df)
    pred_1 = lgb_model_1.predict(test_df)
    pred_2 = lgb_model_2.predict(test_df)
    pred_3 = lgb_model_3.predict(test_df)
    pred_4 = lgb_model_4.predict(test_df)
    
    pred_lgb0 = lgb_0.predict(test_df)
    pred_lgb1 = lgb_1.predict(test_df)
    pred_lgb2 = lgb_2.predict(test_df)
    pred_lgb3 = lgb_3.predict(test_df)
    pred_lgb4 = lgb_4.predict(test_df)
    
    pred_lgb = (pred_0 + pred_1 + pred_2 + pred_3 + pred_4 + pred_lgb0 + pred_lgb1 + pred_lgb2 + pred_lgb3 + pred_lgb4) / 10
    pred_nn = (pred_nn0 + pred_nn1 + pred_nn2 + pred_nn3 + pred_nn4 + pred_nn_0 + pred_nn_1 + pred_nn_2 + pred_nn_3 + pred_nn_4) / 10
    sample_prediction_df['target'] = (pred_lgb+pred_nn)/2
    env.predict(sample_prediction_df) 

In [ ]:
sample_prediction_df